# Análisis de similaridad de los datos de convocantes del ficheros convocatorias
En este notebook vamos a hacer un análisis de similaridad de las columnas de convocantesN1,N2,N3 del fichero convocatorias, disponibles en https://github.com/JaimeObregon/subvenciones/tree/main/files

En el canal de #extración-limpieza de Discord se ha planteado la posibilidade de hacer un análisis de similaridad entre cadenas, para homogeneizar los datos. Se han localizado al menos los siguientes casos de cadenas similares en el dataset (¡Gracias Mireia!):
- RIOJA y LA RIOJA
- COMRCA DE BAJO ARAGÓN-CASPE/BAIX ARAGÓ y COMRCA DE BAJO ARAGÓN

Para localizar cadenas similares se ha propuesto el siguiente recurso: https://towardsdatascience.com/text-similarity-w-levenshtein-distance-in-python-2f7478986e75
También hemos revisado: https://towardsdatascience.com/fuzzywuzzy-find-similar-strings-within-one-column-in-a-pandas-data-frame-99f6c2a0c212

El objetivo de este cuaderno será generar unos ficheros de salida cuyos campos de salida para las columnas de convocantes sean lo más homogeneas posibles.

De esta forma podremos agrupar los resultados de los convocantes, en base a los ficheros que tenemos. Así cuando alguien busque convocatorias lanzadas por "La Rioja" se devolverán los resultados correctos, tanto si en el fichero aparece "LA RIOJA" o "RIOJA" como convocantes. Como En el fichero de convocatorias no aparece el identificador de las entidades, debemos agruparlos en base a su descripción.

Nota: Si estás en Linux/MacOS te tocará cambiar las rutas para cargar los ficheros de \ a /


In [2]:
import pandas as pd
import datetime as dt
from fuzzywuzzy import process, fuzz


# Si estás usando Linux o Mac, aquí es donde tienes que cambiar los separadores de directorio para que te funcione
# Formato Mac - Descomenta estas lineas
# juridicas_1_file = '../files/juridicas_1.csv.gz'
# juridicas_2_file = '../files/juridicas_2.csv.gz' 
#
# Formato Windows
convocatorias_file = '..\\files\\convocatorias.csv.gz'

convocatorias_df = pd.read_csv(convocatorias_file, header=None)

convocatorias_df.head(-10)


C:\Users\Mi PC\AppData\Local\Programs\Python\Python39\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
C:\Users\Mi PC\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (1,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,819060,617500,NO,DIPUTACIÓN PROV. DE JAÉN,DIPUTACIÓN PROVINCIAL DE JAÉN,NaN,25/03/2022,ADQUISICION DE MOBILIARIO PARA EDIFICIO AULAS ...,https://www.dipujaen.es/portal-de-transparenci...,NaN,NaN,350078,1
1,819059,617499,NO,REGIÓN DE MURCIA,INSTITUTO MURCIANO DE ACCION SOCIAL (I.M.A.S.),NaN,25/03/2022,MANCOMUNIDAD RIO MULA MANTENIMIENTO CENTRO ATE...,https://www.borm.es/services/anuncio/ano/2022/...,NaN,NaN,350078,2
2,819058,617498,NO,ONDA,AYUNTAMIENTO DE ONDA,NaN,25/03/2022,Convenio de colaboración entre el Ayuntamiento...,https://www.onda.es/ond/web_php/index.php?cont...,NaN,NaN,350078,3
3,819057,617497,NO,TORRENT,AYUNTAMIENTO DE TORRENT,NaN,25/03/2022,SUBVENCION DIRECTA VARIOS ENTES 2020,https://www.torrent.es/torrentPublic/inicio/se...,NaN,NaN,350078,4
4,819056,617496,NO,REGIÓN DE MURCIA,INSTITUTO MURCIANO DE ACCION SOCIAL (I.M.A.S.),NaN,25/03/2022,AYUNTAMIENTO YECLA MANTENIMIENTO CENTRO ATENCI...,https://www.borm.es/services/anuncio/ano/2022/...,NaN,NaN,350078,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
350063,75968,201020084326021D,NO,ANDALUCÍA,SECRETARÍA GENERAL TÉCNICA (JUSTICIA E INTERIOR),NaN,12/01/2011,ORDEN DE 2 DE DICIEMBRE D 2009,Información no disponible en BDNS2007,NaN,NaN,350078,350064
350064,64666,20092009176181O,NO,ILLES BALEARS,SERVICIO DE OCUPACIÓN DE LAS ISLAS BALEARES (S...,NaN,07/01/2011,CONVOCATÒRIA PREFERENTMENT DESOCUPATS 2009,Información no disponible en BDNS2007,NaN,NaN,350078,350065
350065,76886,201020093744041D,NO,ANDALUCÍA,D. G. DE DESARROLLO SOSTENIBLE DEL MEDIO RURAL,NaN,18/10/2010,Convocatoria y participación de los Grupos de ...,www.juntadeandalucia.es/boja,NaN,NaN,350078,350066
350066,84221,2010201087671O,NO,ILLES BALEARS,SERVICIO DE OCUPACIÓN DE LAS ISLAS BALEARES (S...,NaN,05/10/2010,SUBVENCIONS ACCIONS FORMATIVES PREFERENT DESOC...,Información no disponible en BDNS2007,NaN,NaN,350078,350067


In [4]:
nombres_columnas = [
  "IDConv", #0
  "id", #1
  "mrr", #2
  "convocanteN1", #3
  "convocanteN2", #4
  "convocanteN3", #5
  "fechareg", #6
  "titulo", #7
  "bbreguladoras", #8 - nombre tomado de los ficheros de jurídicas
  "tituloleng", #9
  "verConcesiones", #10 - valores nulos
  "dummy1", #11 - valor 350078 fijo
  "dummy2"  #12 - valor igual a IDConv +1
]

etiquetas_columnas = [
  "IDConv",
  "Código BDNS",
  "MRR",
  "Administración",
  "Departamento",
  "Órgano",
  "Fecha de registro",
  "Título de la convocatoria",
  "URL de las BBRR",
  "Título cooficial",
  "Ver concesiones",
  "dummy1",
  "dummy2",
]

# Eliminamos los tres últimos elementos de la lista de nombres_columnas
columnas_utiles = nombres_columnas[: -3]

print(columnas_utiles)

['IDConv', 'id', 'mrr', 'convocanteN1', 'convocanteN2', 'convocanteN3', 'fechareg', 'titulo', 'bbreguladoras', 'tituloleng']


In [7]:
df = pd.read_csv(convocatorias_file,
names = nombres_columnas,
index_col="IDConv",
dtype={
    "IDConv": str,
    "detalles": object,
},
usecols=columnas_utiles,
parse_dates=["fechareg"],
date_parser=lambda d: dt.datetime.strptime(d, "%d/%m/%Y"),
).sort_values("fechareg")
df.head()

,id,mrr,convocanteN1,convocanteN2,convocanteN3,fechareg,titulo,bbreguladoras,tituloleng
IDConv,,,,,,,,,
49114,200820085002270F,NO,CANTABRIA,SERVICIO CÁNTABRO DE EMPLEO,NaN,2008-06-24,CENTROS ESPECIALES DE EMPLEO: Proyectos Genera...,Información no disponible en BDNS2007,NaN
34230,200820075014000F,NO,CANTABRIA,DIRECCION DEL ICASS,NaN,2008-08-22,PROMOCION SOCIAL SINDICAL,Información no disponible en BDNS2007,NaN
49122,200820085002350F,NO,CANTABRIA,SERVICIO CÁNTABRO DE EMPLEO,NaN,2008-10-08,"Decreto 33/2008, de 3 de abril, por el que se ...",Información no disponible en BDNS2007,NaN
49121,200820085002340F,NO,CANTABRIA,SERVICIO CÁNTABRO DE EMPLEO,NaN,2008-10-08,"Decreto 34/2008, de 3 de abril, por el que se ...",Información no disponible en BDNS2007,NaN
32279,2007200763520703,NO,PONFERRADA,AYUNTAMIENTO DE PONFERRADA,NaN,2009-05-01,CONVOCATORIA SUBVENCIONES A ASOCIACIONES PARA ...,Información no disponible en BDNS2007,NaN


# Primeras pruebas con FuzzyWuzzy
Ahora que tenemos el dataframe, vamos a ver si somos capaces de detectar los casos que se han encontrado manualmente.

Para ello, vamos a quedarnos con los valores únicos de la columna 1 y vamos a ordenarlos. A continuación generaremos nuestra primera prueba.

In [13]:
convocantesN1_unicos = df['convocanteN1'].unique().tolist()
process.extract('LA RIOJA', convocantesN1_unicos,
scorer=fuzz.token_sort_ratio)

[('LA RIOJA', 100),
 ('RIBA, LA', 80),
 ('RODA, LA', 80),
 ('RIOJA', 77),
 ('LA RÀPITA', 75)]

Aquí podemos detectar un problema, ya que RIOJA aparece cuando el grado de similitud es 77, pero antes nos encontramos con 'RIBA, LA' y 'RODA, LA' con una similitud de 80. 

Si le decimos en el programa que nos sustituya aquellos similares a "LA RIOJA" por los que tengan más de un 75% de similitud, estaremos corrigiendo un dato y cambiando erroneamente dos.

En este ejemplo hemos usado uno de los métodos que ofrece FuzzyWuzzy, llamado Token Set Ratio. Esta librería también nos ofrece otro medtodo adicional, conocido como Token Set Ratio. Vamos a probarlo también con el mismo dataset.

In [12]:
process.extract('LA RIOJA', convocantesN1_unicos,
scorer=fuzz.token_set_ratio)

[('LA RIOJA', 100),
 ('RIOJA', 100),
 ('RIBA, LA', 80),
 ('RODA, LA', 80),
 ('BAÑOS DE RIOJA', 77)]

En este caso, encontramos un match del 100% entre Rioja y La Rioja, algo que aparenta ser positivo. Pero veremos más adelante que no lo es, ya que intentaremos quitar de nuestra búsqueda aquellas cadenas que tengan un 100% de similaridad con sigo misma.

Vamos a probar con el otro ejemplo: 'COMARCA DE BAJO ARAGÓN-CASPE/BAIX ARAGÓ'

In [18]:
process.extract('COMARCA DE BAJO ARAGÓN-CASPE/BAIX ARAGÓ', convocantesN1_unicos,
scorer=fuzz.token_sort_ratio)

[('COMARCA DE BAJO ARAGÓN-CASPE/BAIX ARAGÓ-CASP', 94),
 ('COMARCA DE BAJO ARAGÓN', 72),
 ('COMARCA DE BAJO CINCA/BAIX CINCA', 67),
 ('CONSELL COMARCAL DE BAIX CAMP', 58),
 ('COMARCA DE ARANDA', 56)]

In [19]:
process.extract('COMARCA DE BAJO ARAGÓN-CASPE/BAIX ARAGÓ', convocantesN1_unicos,
scorer=fuzz.token_set_ratio)

[('ARAGÓN', 100),
 ('CASPE', 100),
 ('COMARCA DE BAJO ARAGÓN', 100),
 ('COMARCA DE BAJO ARAGÓN-CASPE/BAIX ARAGÓ-CASP', 100),
 ('COMARCA DE BAJO CINCA/BAIX CINCA', 87)]

Probemos con la cadena más corta ''COMARCA DE BAJO ARAGÓN'

In [20]:
process.extract('COMARCA DE BAJO ARAGÓN', convocantesN1_unicos,
scorer=fuzz.token_sort_ratio)

[('COMARCA DE BAJO ARAGÓN', 100),
 ('COMARCA DE ARANDA', 84),
 ('COMARCA DE BAJO MARTÍN', 71),
 ('COMARCA DE BAJO ARAGÓN-CASPE/BAIX ARAGÓ-CASP', 67),
 ('COMARCA DE BAJO CINCA/BAIX CINCA', 64)]

In [21]:
process.extract('COMRCA DE BAJO ARAGÓN', convocantesN1_unicos,
scorer=fuzz.token_set_ratio)

[('ARAGÓN', 100),
 ('COMARCA DE BAJO ARAGÓN', 98),
 ('COMARCA DE ARANDA', 81),
 ('COMARCA DE BAJO ARAGÓN-CASPE/BAIX ARAGÓ-CASP', 79),
 ('COMARCA DE BAJO CINCA/BAIX CINCA', 74)]

# Aplicación de la técnica al conjunto completo
Para esta sección vamos a probar las técnicas que comparte Thanh Huynh en su publicación en TowardsDataScience. Adaptamos el código a nuestro ejemplo

In [22]:
#Create tuples of brand names, matched brand names, and the score
score_sort = [(x,) + i
             for x in convocantesN1_unicos 
             for i in process.extract(x, convocantesN1_unicos, scorer=fuzz.token_sort_ratio)]
#Create a dataframe from the tuples
similarity_sort = pd.DataFrame(score_sort, columns=['brand_sort','match_sort','score_sort'])
similarity_sort.head()

,brand_sort,match_sort,score_sort
0,CANTABRIA,CANTABRIA,100
1,CANTABRIA,CANARIAS,82
2,CANTABRIA,CANDELARIA,74
3,CANTABRIA,CABRA,71
4,CANTABRIA,CHANTADA,71


In [23]:
import numpy as np
similarity_sort['sorted_brand_sort'] = np.minimum(similarity_sort['brand_sort'], similarity_sort['match_sort'])
similarity_sort.head()

,brand_sort,match_sort,score_sort,sorted_brand_sort
0,CANTABRIA,CANTABRIA,100,CANTABRIA
1,CANTABRIA,CANARIAS,82,CANARIAS
2,CANTABRIA,CANDELARIA,74,CANDELARIA
3,CANTABRIA,CABRA,71,CABRA
4,CANTABRIA,CHANTADA,71,CANTABRIA


In [24]:
high_score_sort = similarity_sort[(similarity_sort['score_sort'] >= 80) &
                (similarity_sort['brand_sort'] !=  similarity_sort['match_sort']) &
                (similarity_sort['sorted_brand_sort'] != similarity_sort['match_sort'])]
high_score_sort = high_score_sort.drop('sorted_brand_sort',axis=1).copy()

In [25]:
high_score_sort.groupby(['brand_sort','score_sort']).agg(
                        {'match_sort': ', '.join}).sort_values(
                        ['score_sort'], ascending=False)

,,match_sort
brand_sort,score_sort,
PALS,100,PAÜLS
AGUAVIVA,94,AIGUAVIVA
BENISSA,93,BENISSANÓ
ALFAFAR,93,ALFAFARA
CONSELL COMARCAL DE VALLÈS OCCIDENTAL,93,CONSELL COMARCAL DEL VALLÈS ORIENTAL
...,...,...
OLVERA,80,VERA
ONDA,80,ONDARA
ALMUDÉVAR,80,ALMUÑÉCAR


In [27]:
resultados_token_sort_N1 = high_score_sort.groupby(['brand_sort','score_sort']).agg(
                        {'match_sort': ', '.join}).sort_values(
                        ['score_sort'], ascending=False).to_csv('resultados_token_sort_N1.csv')

Vamos a probar ahora la versión de Token Set Ratio, disponible en: https://github.com/thuynh323/Natural-language-processing/blob/master/FuzzyWuzzy%20-%20Ramen%20Rater%20List/Find%20similar%20strings%20with%20FuzzyWuzzy.ipynb

Para ello tan sólo tenemos que cambiar el parámetro de unique_brand por nuestro listado de convocantesN1_unicos:

In [26]:
#Create tuples of brand names, matched brand names, and the score
score_set = [(x,) + i
             for x in convocantesN1_unicos 
             for i in process.extract(x, convocantesN1_unicos, scorer=fuzz.token_set_ratio)]
#Create dataframe from the tuples and derive representative values
similarity_set = pd.DataFrame(score_set, columns=['brand_set','match_set','score_set'])
similarity_set['sorted_brand_set'] = np.minimum(similarity_set['brand_set'], similarity_set['match_set'])

#Pick values
high_score_set = similarity_set[(similarity_set['score_set'] >= 80) & 
                                    (similarity_set['brand_set'] != similarity_set['match_set']) & 
                                    (similarity_set['sorted_brand_set'] != similarity_set['match_set'])]

#Drop the representative value column
high_score_set = high_score_set.drop('sorted_brand_set',axis=1).copy()

#Group brands by matches and scores
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
high_score_set.groupby(['match_set','score_set']).agg(
                       {'brand_set': ', '.join}).sort_values(
                       ['score_set'], ascending=False)

,,brand_set
match_set,score_set,
COMARCA DE BAJO ARAGÓN,100,ARAGÓN
PALMA DEL RÍO,100,PALMA
PALMA,100,"CABILDO INSULAR DE LA PALMA, FUENCALIENTE DE LA PALMA"
COMARCA DE COMUNIDAD DE CALATAYUD,100,CALATAYUD
COMARCA DE SOMONTANO DE BARBASTRO,100,BARBASTRO
PALENCIA,100,DIPUTACIÓN PROV. DE PALENCIA
PADERNE DE ALLARIZ,100,PADERNE
OURENSE,100,DIPUTACIÓN PROV. DE OURENSE
"OLIVA, LA",100,"OLIVA, OLIVA DE LA FRONTERA"


In [28]:
resultados_token_set_N1 = high_score_set.groupby(['match_set','score_set']).agg(
                       {'brand_set': ', '.join}).sort_values(
                       ['score_set'], ascending=False).to_csv('resultados_token_set_N1.csv')